In [116]:
from arcgis import GIS,mapping,features,geometry
import arcpy 
import json
import sys 
import requests

In [117]:
#To start the trace we first need to compile a few parameters. 
# Portal URL 
pUrl = "https://water.bd.esri.com/portal"
gis = GIS(pUrl,"ic_water","#######")
print (gis.url)

https://water.bd.esri.com/portal


In [118]:
# The next step is to get the GPS starting point.We assume GPS data comes in WGS84 projection
# for the purpose of the notebook we have a smaple start point

inPt = [float(i) for i in arcpy.GetParameterAsText(0).split(",")] if not arcpy.GetParameterAsText(0)== "" else [-88.122724, 41.7731766]
print(inPt)

[-88.122724, 41.7731766]


In [119]:
# Utility Network base service URL
sUrl = "https://water-un.bd.esri.com/server/rest/services/Naperville/Water_Distribution_Utility_Network"

# The UN trace has several input options but the quickest way to set a start point is on a line segment.
# Need to query the line layer to get the start feature. Check the REST page of the service to get the correct layer ID
lLayer = features.FeatureLayer(sUrl + "/FeatureServer/515",gis)
print (lLayer.properties.name)




Water Line


In [120]:
# To query the service, construct the correct REST format
pt = {"x" : inPt[0], "y" : inPt[1]}

# Since this is GPS data, it is unlikely to be right on the line...
# we want to progressively increase our search distance until a line is found:
line = None
d=20
while line is None and d<=200:
    res = lLayer.query(outFields="globalid",geometry=pt,geometryType="esriGeometryPoint",inSR=4326,spatialRel="esriSpatialRelIntersects",distance=d,units="esriSRUnit_Foot",returnGeometry=True,outSR=4326)
    if len(res.features)>0: line = res.features[0] # This is the first, not necessarily the closeset line!
    else: d+=20
if not line: raise Exception("No Line Found")
else: print(line)

{"geometry": {"paths": [[[-88.12286030642272, 41.773047352552396], [-88.12278921932774, 41.77306411313595], [-88.1226244850675, 41.77310421047509], [-88.12253268950369, 41.77311818320672], [-88.1223821375177, 41.77313856033188], [-88.12237070867023, 41.77314010688092], [-88.12229237372952, 41.773148001467625], [-88.12222249886871, 41.77315152227112], [-88.12206026588429, 41.77315969648341], [-88.121918388035, 41.77316392741448], [-88.12164916661115, 41.77317195814588]]]}, "attributes": {"globalid": "{4DFC2E5F-AE41-4C07-8C77-BC62B8FBE287}"}}


In [121]:
# Now that we have a line we need to idenify where on the line the input point is closest to (% along line)
# We can construct a arcgis geometry to do this.
line.geometry["spatialReference"]= {"wkid" : 4326}
ln = geometry.Geometry(line.geometry)

# Convert the start point to a geometry 
pt["spatialReference"]= {"wkid" : 4326}
pt = geometry.Geometry(pt)

print("{},{}".format(ln.is_valid(),pt.is_valid()))

True,True


In [122]:
# Use the geometries to find the % measure along the line
measure_start = ln.query_point_and_distance(pt,True)
print("\t\tpt on line\t\t\t\t percent along line\t distance to line\n{}\n".format(measure_start))



		pt on line				 percent along line	 distance to line
(<PointGeometry object at 0x200d4042710[0x200d09f7328]>, 0.13347064322120789, 9.387131011558406e-05, False)



In [123]:
# To define the start for the trace, we need the line GLOBALID and the distance along the line.
start = [line.attributes['globalid'],measure_start[1]]
print (start)

['{4DFC2E5F-AE41-4C07-8C77-BC62B8FBE287}', 0.13347064322120789]


In [124]:
# set up the trace
unUrl = sUrl + "/UtilityNetworkServer/trace"
data = {} 
data["gdbVersion"] = "sde.DEFAULT"
data["sessionId"] = ""
data["moment"] = ""
data["traceType"] = "isolation"
isolated = "false" # Only return barriers
data["f"] = "json" # Output json format
data["token"] = gis._con._token 

# Parse start into the correct format. You can get the parameters from a fiddler log of a trace from Pro or the documentation
data["traceLocations"] = """[{{"traceLocationType":"startingPoint","globalId":"{}","percentAlong":{}}}]""".format(start[0],start[1])

# a big json parameter is needed here. You can get the parameters from a fiddler log of a trace from Pro or review the documentation
data["traceConfiguration"] = """{"includeContainers":true,"includeContent":false,"includeStructures":true,"includeBarriers":true,"validateConsistency":true,"includeIsolated":false,"ignoreBarriersAtStartingPoints":false,"domainNetworkName":"water","tierName":"water system","targetTierName":"","subnetworkName":"","diagramTemplateName":"","shortestPathNetworkAttributeName":"","filterBitsetNetworkAttributeName":"","traversabilityScope":"junctionsAndEdges","conditionBarriers":[{"name":"P:Device Status","type":"networkAttribute","operator":"equal","value":0,"combineUsingOr":true,"isSpecificValue":true},{"name":"Lifecycle Status","type":"networkAttribute","operator":"doesNotIncludeAny","value":24,"combineUsingOr":true,"isSpecificValue":true},{"name":"Category","type":"category","operator":"equal","value":"CP Only","combineUsingOr":false,"isSpecificValue":true}],"functionBarriers":[],"arcadeExpressionBarrier":"","filterBarriers":[{"name":"Operable","type":"networkAttribute","operator":"equal","value":1,"combineUsingOr":false,"isSpecificValue":true}],"filterFunctionBarriers":[],"filterScope":"junctionsAndEdges","functions":[],"nearestNeighbor":{"count":-1,"costNetworkAttributeName":"","nearestCategories":[],"nearestAssets":[]},"outputFilters":[],"outputConditions":[],"propagators":[]}"""

# request the trace
res = requests.get(unUrl,params=data)
# The trace results are only the barriers identified during the isolation trace
# The result is an unordered list of  features with a limited set of attributes
print (res.text)


{"traceResults":{"elements":[{"networkSourceId":6,"globalId":"{93AF5D5A-14FF-47BD-86B8-A773EEC7EA1B}","objectId":11172,"terminalId":1,"assetGroupCode":2,"assetTypeCode":170},{"networkSourceId":6,"globalId":"{197A5573-CB5E-4C64-A86A-473ED3D33408}","objectId":11469,"terminalId":1,"assetGroupCode":2,"assetTypeCode":170},{"networkSourceId":6,"globalId":"{10288BB8-2E93-42A1-B823-C7793A340F5F}","objectId":12155,"terminalId":1,"assetGroupCode":2,"assetTypeCode":170},{"networkSourceId":6,"globalId":"{BB176677-30E7-4D34-BC6F-88ACDB820623}","objectId":12116,"terminalId":1,"assetGroupCode":2,"assetTypeCode":170},{"networkSourceId":6,"globalId":"{96D1F6FA-4765-4A67-BA2A-0E5F04B164A0}","objectId":11893,"terminalId":1,"assetGroupCode":2,"assetTypeCode":170}],"diagramName":"\u003cplaceholder\u003e","globalFunctionResults":[],"kFeaturesForKNNFound":false,"startingPointsIgnored":false,"warnings":[]},"success":true}


To get meters we need to make very slight modifications to the **trace configuration.**

Change the following parameters:
1. The parameter to return not just the barriers, but all of the network elements that are impacted by the isolation
        "includeIsolated":true
2. The parameter to apply a filter the elements being returned to just the service connections asset group

        "outputFilters":[{"name":"Device Asset Group","type":"networkAttribute","operator":"equal","value":12,"combineUsingOr":false,"isSpecificValue":true}]

**Making these changes to the previous trace configuration will result in only service connections being returned from the trace.**

Now back to the isolation valve results...

In [ ]:
# While useful, we need to go back and query the service for the other valve fields we cae about 
fields = ["assetgroup","assettype","Operable","assetid","designtype","designinfo","additionaldetails", "objectid"]
# Need to query the device layer to get the start feature. Check the REST page of the service to get the correct layer ID
dLayer = features.FeatureLayer(sUrl + "/FeatureServer/501",gis)
# Make query using objectids from trace results 
devices = json.loads(res.content)['traceResults']['elements']
query = """ operable = 1  and objectid in ({})""".format(",".join([str(i['objectId']) for i in devices]))

# Query the service 
valves = dLayer.query(query,out_fields=fields,return_geometry=True,outSR=4326)
print(valves)

In [126]:
# While more useful, we need to translate the coded values to descriptions to make the data human readable
# to translate them we need a domain lookup. The layer properties has ALL the domains we need.
#Some domains are described in the layer properties at the field level

dOperable = {i["code"]:i["name"] for i in  [i["domain"]["codedValues"] for i in  dLayer.properties.fields if i["name"]=="operable"][0]}

# Some domains are assigned at the subtype level so each feature subtype ("AssetGroup") needs to be considered:
for feature in valves.features:
    domains = [s.domains for s in dLayer.properties.subtypes if s["code"]== feature.attributes["assetgroup"]][0]
    try: # in case of data anomalies
        feature.attributes["assetgroup"] = [i.name for i in dLayer.properties.subtypes if i.code==feature.attributes["assetgroup"]][0]
        feature.attributes["operable"] = dOperable[feature.attributes["operable"]][0]
    except:pass
    try:
        feature.attributes["assettype"] = [i.name for i in domains.assettype.codedValues if i.code==feature.attributes["assettype"]][0]
    except:pass 
    try:
        feature.attributes["designtype"] = [i.name for i in domains.designtype.codedValues if i.code==feature.attributes["designtype"]][0]
    except: pass 
    try:
        feature.attributes["additionaldetails"] = [i.name for i in domains.additionaldetails.codedValues if i.code==feature.attributes["additionaldetails"]][0]
    except:pass

for v in valves: print(v.attributes)


{'assetgroup': 'Controllable Valve', 'assettype': 'System', 'operable': 'T', 'assetid': 'cv-9818', 'designtype': 'Gate', 'designinfo': 12.0, 'additionaldetails': 'Clockwise', 'objectid': 11172}
{'assetgroup': 'Controllable Valve', 'assettype': 'System', 'operable': 'T', 'assetid': 'cv-10111', 'designtype': 'Gate', 'designinfo': 12.0, 'additionaldetails': 'Clockwise', 'objectid': 11469}
{'assetgroup': 'Controllable Valve', 'assettype': 'System', 'operable': 'T', 'assetid': 'cv-10535', 'designtype': 'Gate', 'designinfo': 12.0, 'additionaldetails': 'Clockwise', 'objectid': 11893}
{'assetgroup': 'Controllable Valve', 'assettype': 'System', 'operable': 'T', 'assetid': 'cv-10758', 'designtype': 'Gate', 'designinfo': 12.0, 'additionaldetails': 'Clockwise', 'objectid': 12116}
{'assetgroup': 'Controllable Valve', 'assettype': 'System', 'operable': 'T', 'assetid': 'cv-10797', 'designtype': 'Gate', 'designinfo': 12.0, 'additionaldetails': 'Clockwise', 'objectid': 12155}


In [ ]:
# There, we now have a set of isolation devices!